In [51]:
from deep_translator import GoogleTranslator
import json
import glob
import re
import os
import pandas as pd



In [2]:
def split_string_into_chunks(text, max_chunk_size=4500):
    # Use regular expressions to split the text into sentences
    # Assumes sentences end with ".", "!", or "?", followed by a space or newline
    sentences = re.split(r'(?<=[.!?])\s+', text)
 
    # Initialize variables
    current_chunk = ""
    chunks = []
 
    # Iterate through sentences and add them to the current chunk
    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chunk_size:
            current_chunk += sentence + " "
        else:
            # If adding the sentence would exceed the chunk size, start a new chunk
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
 
    # Add the last chunk, if any
    if current_chunk:
        chunks.append(current_chunk.strip())
 
    return chunks

In [52]:
def translate(path):
    print(rf'{path}/*.csv')
    for file in glob.glob(rf'{path}/*.csv'):
        print(file)
        df = pd.read_csv(file, index_col="Unnamed: 0", engine="python")
        
        for ind, _ in df.iterrows():
            chunks = split_string_into_chunks(df.at[ind, "text"])
            df.at[ind, "text"] = ' '.join(GoogleTranslator(source='auto', target='en').translate_batch(chunks))
        

translate("data")

data/*.csv
data/articles.csv


RequestError: Request exception can happen due to an api connection error. Please check your connection and try again